# Analyze Cloudflare WAF logs

There are various kinds of useful tasks we can perform to analyze Cloudflare WAF logs.

- **Threat Detection**: Analyze logs for patterns that indicate security threats, such as DDoS attacks, SQL injection attempts, and cross-site scripting (XSS) attacks. Cloudflare logs provide data about the threat landscape that your infrastructure is exposed to.
- **IP Address Analysis**: Identify suspicious activities by analyzing requests from IP addresses that exhibit unusual behavior, such as a high number of requests or requests that trigger security rules.
- **Rate Limiting and Blocking Events**: Assess the effectiveness of security rules and rate limits by analyzing blocked or challenged requests.

Install dependencies.

In [ ]:
%pip install https://scanner-dev-public.s3.us-west-2.amazonaws.com/sdks/python/scanner_client-0.0.1-py3-none-any.whl

In [ ]:
%pip install seaborn matplotlib yfiles_jupyter_graphs

Import libraries.

In [ ]:
from scanner_client import Scanner
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime, timezone, timedelta
import pandas as pd
from yfiles_jupyter_graphs import GraphWidget
import os

pd.set_option('max_colwidth', None)

def convert_results_to_data_frame(results):
    rows = [row.columns.to_dict() for row in results.rows]
    column_tags = results.column_tags.to_dict()
    if len(column_tags) > 0:
        # If this is a table, use the column ordering in the data frame
        return pd.DataFrame(data=rows, columns=results.column_ordering)
    else:
        # Otherwise, this is a list of log events, so use pandas JSON
        # normalization to set the table columns to the union of all keys.
        return pd.json_normalize(rows)

Initialize Scanner API client:

In [ ]:
scanner = Scanner(
    api_url=os.environ["SCANNER_API_URL"],
    api_key=os.environ["SCANNER_API_KEY"],
)

Set analyzed time range to be the last 7 days.

In [ ]:
end_time = datetime.now(tz=timezone.utc)
start_time = end_time - timedelta(days=7)

Run a Scanner query to look for possible SQL injection attacks that were blocked by Cloudflare.

We start by looking for the presence of various kinds of SQL syntax in request query parameters.

In [ ]:
response = scanner.query.blocking_query(
    start_time=start_time.isoformat(),
    end_time=end_time.isoformat(),
    query_text="""
        %ingest.source_type: 'cloudflare:firewall'
        ClientRequestQuery: (
          'union*select' or
          'select*from' or
          'drop*table' or
          'insert*into' or
          'update*set' or
          'varchar'
        )
        Action: skip
        | stats 
          min(timestamp) as firstTime,
          max(timestamp) as lastTime,
          countdistinct(ClientRequestUserAgent) as numDistinctUserAgentsUsed,
          countdistinct(ClientRequestQuery) as numDistinctRequestQueriesUsed,
          ClientRequestUserAgent as randomUserAgent,
          ClientRequestQuery as randomRequestQuery
          by ClientIP
    """,
)

Number of rows in response:

In [ ]:
len(response.results.rows)

Transform Scanner search results into a `pandas` data frame, and show the first few rows:

In [ ]:
possible_sql_attack_df = convert_results_to_data_frame(response.results)
possible_sql_attack_df.head()

Show a sample of request queries to validate whether these indeed look like SQL injection attacks.

Note that these have all been stopped by Cloudflare because we selected for events with `Action: skip`, but they are spooky nonetheless.

In [ ]:
possible_sql_attack_df[['randomRequestQuery']].head()

Get the most prolific IP addresses performing these SQL injection requests.

In [ ]:
prolific_ips_df = possible_sql_attack_df.groupby("ClientIP")["@q.count"]\
    .sum()\
    .sort_values(ascending=False)\
    .reset_index()

prolific_ips_df.head()

Render a chart of the most prolific IP addresses performing SQL injection requests.

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=prolific_ips_df[:10], y='ClientIP', x='@q.count')
plt.title('Top 10 Most Prolific Client IPs Performing SQL Injection Requests')
plt.xlabel('Frequency')
plt.ylabel('Client IP Addresses')
plt.show()

Take the top 3 most prolific ip addresses and query for their activity to find more results.

In [ ]:
top_prolific_client_ips = list(prolific_ips_df['ClientIP'])[0:3]
top_prolific_client_ips

In [ ]:
response = scanner.query.blocking_query(
    start_time=start_time.isoformat(),
    end_time=end_time.isoformat(),
    query_text=f"""
        %ingest.source_type: 'cloudflare:firewall'
        ClientIP: ({' or '.join(top_prolific_client_ips)})
        | stats
          min(timestamp) as firstTime,
          max(timestamp) as lastTime
          by 
          ClientIP,
          ClientRequestHost,
          Action
    """,
)

Number of rows in response:

In [ ]:
len(response.results.rows)

Convert results to a data frame:

In [ ]:
activity_df = convert_results_to_data_frame(response.results)
activity_df.head()

In [ ]:
activity_df['ActivityLabel'] = activity_df['ClientIP'] + ' - ' +\
    activity_df['ClientRequestHost'] + ' - ' +\
    activity_df['Action']

activity_df.head()

Make a chart of top activity that was skipped by Cloudflare.

In [ ]:
skipped_activity_df = activity_df[activity_df["Action"] == 'skip']

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=skipped_activity_df[:10], y='ActivityLabel', x='@q.count', color="lightcoral")
plt.title('Prolific IPs - Activity Skipped by Cloudflare')
plt.xlabel('Frequency')
plt.ylabel('Activity')
plt.show()

Make a chart of top activity that was *not* skipped by Cloudflare.

In [ ]:
non_skipped_activity_df = activity_df[activity_df["Action"] != 'skip']

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=non_skipped_activity_df[:10], y='ActivityLabel', x='@q.count')
plt.title('Prolific IPs - Activity NOT Skipped by Cloudflare')
plt.xlabel('Frequency')
plt.ylabel('Activity')
plt.show()

Visualize the prolific IP addresses and the hosts and URL paths they connect to.

First, create graph nodes and edges.

In [ ]:
top_activity_df = activity_df[0:100]
client_ips = top_prolific_client_ips

group_by_client_ip_df = top_activity_df.groupby('ClientIP')["@q.count"]\
    .sum()\
    .sort_values(ascending=False)\
    .reset_index()
min_request_count_for_client_ip = group_by_client_ip_df['@q.count'].min()
max_request_count_for_client_ip = group_by_client_ip_df['@q.count'].max()


client_request_hosts = set(top_activity_df['ClientRequestHost'])
group_by_host_df = top_activity_df.groupby('ClientRequestHost')["@q.count"]\
    .sum()\
    .sort_values(ascending=False)\
    .reset_index()
min_request_count_for_host = group_by_host_df['@q.count'].min()
max_request_count_for_host = group_by_host_df['@q.count'].max()

min_request_count_for_node = min(min_request_count_for_client_ip, min_request_count_for_host)
max_request_count_for_node = max(max_request_count_for_client_ip, max_request_count_for_host)


In [ ]:
MIN_SCALE = 1.0
MAX_SCALE = 10.0

def get_node_scale_factor(num_requests):
    numer = num_requests - min_request_count_for_node
    denom = max_request_count_for_node - min_request_count_for_node
    if denom == 0:
        return 1.0
    scale_factor = MIN_SCALE + (numer / denom) * (MAX_SCALE - MIN_SCALE)
    return scale_factor

nodes = []
for client_ip in client_ips:
    num_requests = top_activity_df[top_activity_df['ClientIP'] == client_ip]["@q.count"].sum()
    num_hosts = len(set(top_activity_df[top_activity_df['ClientIP'] == client_ip]["ClientRequestHost"]))
    node_scale_factor = get_node_scale_factor(num_requests)
    nodes.append({
        'id': client_ip,
        'properties': {
            'label': f"Client IP: {client_ip}",
            'client_ip': client_ip,
            'num_requests_sent': num_requests,
            'num_hosts_connected_to': num_hosts,
            'node_scale_factor': node_scale_factor,
        },
    })

for client_request_host in client_request_hosts:
    num_requests = top_activity_df[top_activity_df['ClientRequestHost'] == client_request_host]["@q.count"].sum()
    node_scale_factor = get_node_scale_factor(num_requests)
    nodes.append({
        'id': client_request_host,
        'properties': {
            'label': f"Host: {client_request_host}",
            'host': client_request_host,
            'num_requests_received': num_requests,
            'node_scale_factor': node_scale_factor,
        },
    })

activity_rows = top_activity_df.to_dict(orient='records')
edges = []
for i, activity_row in enumerate(activity_rows):
    edge_id = i
    client_ip = activity_row.get('ClientIP')
    client_request_host = activity_row.get('ClientRequestHost')
    action = activity_row.get('Action')
    first_time = activity_row.get('firstTime')
    last_time = activity_row.get('lastTime')
    count = int(activity_row.get('@q.count'))
    edges.append({
        'id': edge_id,
        'start': client_ip,
        'end': client_request_host,
        'properties': {
            'source_client_ip': client_ip,
            'target_request_host': client_request_host,
            'action': action,
            'first_time': first_time,
            'last_time': last_time,
            'count': count,
            'label': f"{action}: {count}",
        },
    })

def custom_node_scale_factor_mapping(node):
    return node.get('properties', {}).get('node_scale_factor', 1.0)

Generate interactive graph visualization of `AssumeRole` chains.

- Click and drag to navigate. Use mouse wheel to zoom in/out.
- Click on a node or an edge to select it and see what it is connected to.
- When a node or edge is selected, inspect its properties in the `Data` tab in the side bar.
- Search for a node or edge via the `Search` tab in the side bar.
- Change the layout of the graph to examine relationships in different ways.

In [ ]:
w = GraphWidget()
w.nodes = nodes
w.edges = edges
w.directed = True
w.set_node_scale_factor_mapping(custom_node_scale_factor_mapping)
w